`conda activate spaGAN`

In [2]:
import numpy as np
import json
import scipy
from scipy import sparse as sp
from scipy.sparse import coo_matrix
import pandas as pd
import os
import torch
import pickle

import matplotlib.pyplot as plt

#from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv
#from torch_geometric.utils import train_test_split_edges
from torch_geometric.nn import VGAE
import torch_geometric.transforms as T
from torch_geometric.data import Data

In [3]:
path = '/Volumes/LaCie/school/combine_lab/SpaGE/SpaGE_Datasets/'

gene_express = pd.read_csv(path+'ST_gene_expression.csv')
gene_exp = gene_express.set_index('CellID', drop=True)
gene_exp = gene_exp.drop(['Unnamed: 0','X', 'Y'], axis=1)

adj = pd.read_pickle(path+'adj_mat_crs_220.pickle')

In [4]:
print(gene_exp.shape)
gene_exp

(3405, 33)


,Gad2,Slc32a1,Crhbp,Cnr1,Vip,Cpne5,Pthlh,Crh,Tbr1,Lamp5,...,Ctps,Anln,Mrc1,Hexb,Ttr,Foxj1,Vtn,Flt1,Apln,Acta2
CellID,,,,,,,,,,,,,,,,,,,,,
778,1.621371,1.777401,-1.285498,0.169445,-0.417264,0.932325,-1.285498,-0.635124,1.368875,0.276102,...,0.817961,0.050672,-0.237032,0.876704,-0.237032,-0.417264,-0.635124,-1.285498,-0.237032,-0.417264
3642,2.723553,2.088070,0.162081,-0.609545,-0.353881,-0.609545,-0.609545,-0.150690,0.500071,-0.954585,...,0.162081,-0.150690,-0.609545,-0.150690,-0.609545,-0.609545,-0.353881,-0.954585,0.017943,-0.609545
769,2.459380,2.289477,1.116087,-0.466125,-0.148632,-0.951611,0.186618,-0.466125,1.044207,-0.951611,...,-0.022988,-0.148632,-0.293912,1.150048,-0.677581,-0.148632,-0.022988,-0.951611,-0.466125,-0.293912
5870,2.253989,1.549556,-0.197524,0.225092,-0.510714,-0.982660,0.135831,-0.982660,1.924713,-0.072863,...,0.225092,-0.982660,-0.072863,0.515976,-0.072863,-0.717495,-0.717495,-0.982660,-0.717495,-0.072863
5682,1.496840,2.316222,-0.944582,1.345124,0.013995,-0.944582,-0.944582,-0.944582,0.283120,3.576333,...,0.495107,-0.354909,0.013995,0.283120,-0.354909,-0.354909,-0.354909,-0.354909,0.283120,-0.944582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3635,0.309345,-0.843207,-0.843207,-0.843207,0.309345,0.309345,-0.843207,0.783172,2.051002,2.492857,...,1.309683,-0.843207,1.086314,-0.843207,-0.843207,-0.843207,-0.843207,-0.843207,0.309345,1.086314
3336,1.396286,-0.977202,-0.977202,1.229384,1.229384,-0.977202,-0.977202,-0.977202,0.901510,-0.977202,...,0.381686,0.759097,0.110551,-0.278649,-0.278649,-0.977202,1.024502,0.381686,-0.278649,2.598400
6462,0.232061,-0.423666,-1.342403,-0.423666,-0.156431,-1.342403,-0.423666,0.630742,2.287202,-1.342403,...,0.055889,0.232061,2.117512,-0.784520,0.830838,0.232061,-0.423666,-1.342403,0.232061,1.848433


In [5]:
edges = torch.tensor(adj.toarray()).float()
edges

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [14]:
print(edges[np.where(edges > 0)]) # edge weights for each interaction
print(edges[np.where(edges > 0)].shape) # number of connections

tensor([1., 1., 1.,  ..., 1., 1., 1.])
torch.Size([3824])


# Preprocess

In [6]:
def csr2tensor(csr):
    """convert compressed sparse row format to sparse tensor"""
    coo = csr.tocoo()
    values = coo.data
    indices = np.vstack((coo.row, coo.col))
    
    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = coo.shape
    
    tens = torch.sparse.FloatTensor(i, v, torch.Size(shape))#.to_dense()
    coalesce_tensor = tens.coalesce()
    
    return coalesce_tensor.indices()

In [7]:
A = csr2tensor(adj)
A

tensor([[   0,    1,    3,  ..., 3403, 3404, 3404],
        [2258, 1633, 1608,  ...,  925, 2145, 2294]])

In [8]:
features = torch.tensor(gene_exp.values).float()
features

tensor([[ 1.6214,  1.7774, -1.2855,  ..., -1.2855, -0.2370, -0.4173],
        [ 2.7236,  2.0881,  0.1621,  ..., -0.9546,  0.0179, -0.6095],
        [ 2.4594,  2.2895,  1.1161,  ..., -0.9516, -0.4661, -0.2939],
        ...,
        [ 0.2321, -0.4237, -1.3424,  ..., -1.3424,  0.2321,  1.8484],
        [ 0.6325, -1.0411, -1.0411,  ..., -1.0411,  0.6325,  1.3892],
        [ 0.6470, -1.1493, -1.1493,  ..., -1.1493, -1.1493,  1.5847]])

In [9]:
data = Data(x=features, edge_index=A, edge_attr= edges, requires_grad=True)
data

Data(x=[3405, 33], edge_index=[2, 3824], edge_attr=[3405, 3405], requires_grad=True)

In [10]:
print(data.num_features, data.num_nodes)

33 3405


In [15]:
print(data.x)
print(data.x.shape)
print(data.edge_index)
print(data.edge_index.shape)

tensor([[ 1.6214,  1.7774, -1.2855,  ..., -1.2855, -0.2370, -0.4173],
        [ 2.7236,  2.0881,  0.1621,  ..., -0.9546,  0.0179, -0.6095],
        [ 2.4594,  2.2895,  1.1161,  ..., -0.9516, -0.4661, -0.2939],
        ...,
        [ 0.2321, -0.4237, -1.3424,  ..., -1.3424,  0.2321,  1.8484],
        [ 0.6325, -1.0411, -1.0411,  ..., -1.0411,  0.6325,  1.3892],
        [ 0.6470, -1.1493, -1.1493,  ..., -1.1493, -1.1493,  1.5847]])
torch.Size([3405, 33])
tensor([[   0,    1,    3,  ..., 3403, 3404, 3404],
        [2258, 1633, 1608,  ...,  925, 2145, 2294]])
torch.Size([2, 3824])


In [16]:
print(torch.nonzero(data.x))
# print(data.x[0, 6])

tensor([[   0,    0],
        [   0,    1],
        [   0,    2],
        ...,
        [3404,   30],
        [3404,   31],
        [3404,   32]])


In [12]:
# data = train_test_split_edges(data)
# data

In [17]:
transform = T.RandomLinkSplit(add_negative_train_samples=True)
train_data, val_data, test_data = transform(data)

In [18]:
train_data

Data(x=[3405, 33], edge_index=[2, 2678], edge_attr=[3405, 3405], requires_grad=True, edge_label=[5356], edge_label_index=[2, 5356])

In [19]:
test_data

Data(x=[3405, 33], edge_index=[2, 3060], edge_attr=[3405, 3405], requires_grad=True, edge_label=[1528], edge_label_index=[2, 1528])

In [20]:
train_data.pos_edge_label_index

AttributeError: 'GlobalStorage' object has no attribute 'pos_edge_label_index'

In [ ]:
train_data.edge_index == train_data.pos_edge_label_index

In [ ]:
test_data

In [21]:
class VGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VGCNEncoder, self).__init__()
        """
        First GCN layer generates a lower-dimensional ft matrix;
        Second GCN layer generates mu and log sigma sq.
        """
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logvar = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logvar(x, edge_index)

In [22]:
out_channels = 50 # number of topics/clusters
num_features = data.num_features
epochs = 50
print(num_features)

33


In [24]:
model = VGAE(VGCNEncoder(num_features, out_channels))  

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
# train_data.pos_edge_label_index = train_data.pos_edge_label_index.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [25]:
from torchviz import make_dot

ModuleNotFoundError: No module named 'torchviz'

In [26]:
print(model)

VGAE(
  (encoder): VGCNEncoder(
    (conv1): GCNConv(33, 100)
    (conv_mu): GCNConv(100, 50)
    (conv_logvar): GCNConv(100, 50)
  )
  (decoder): InnerProductDecoder()
)


In [ ]:
# next(iter(data.x))

In [ ]:
# next(iter(data.x)).shape

In [27]:
model_test = VGCNEncoder(num_features, out_channels)

In [ ]:
# # need to have dummy input that can pass thru the model's forward() method.
# batch = next(iter(data.x)).requires_grad_(True)
# batch

In [28]:
train_data.x.shape

torch.Size([3405, 33])

In [29]:
train_data.edge_index.shape

torch.Size([2, 2678])

In [30]:
# return mean and std
yhat = model_test.forward(train_data.x, train_data.edge_index) 
yhat

(tensor([[ 0.5635,  0.1460, -0.5364,  ..., -0.0182, -0.0111, -0.9323],
         [-0.4392, -0.5678, -1.6455,  ..., -0.4338,  0.0179, -0.9101],
         [ 0.3348, -0.2295, -0.7337,  ..., -0.3177,  0.3427, -0.4180],
         ...,
         [ 0.4770,  0.9767, -0.2441,  ...,  0.8915, -0.3157, -0.3199],
         [ 0.7011, -0.5820, -1.0935,  ...,  0.0742, -0.3640, -1.0829],
         [ 0.8474, -0.0194, -1.2681,  ...,  0.2735, -0.4764, -0.4694]],
        grad_fn=<AddBackward0>),
 tensor([[-0.1409, -0.2140, -0.9122,  ...,  0.6018,  0.6206,  0.1975],
         [-0.4021, -0.8107, -0.6132,  ..., -0.2794,  0.3106, -0.6400],
         [ 0.2075, -1.2223, -1.0550,  ...,  0.2082,  0.1204, -0.5063],
         ...,
         [-0.1805,  0.0349, -1.3207,  ...,  0.7696,  0.0980,  0.3758],
         [-0.6132, -0.0652, -0.7839,  ...,  0.1797,  0.2977, -0.2269],
         [-0.1368,  0.5108, -1.1784,  ...,  0.5517,  0.7310, -0.1733]],
        grad_fn=<AddBackward0>))

In [31]:
yhat[0].shape

torch.Size([3405, 50])

In [32]:
make_dot(yhat, params=dict(model_test.named_parameters()))

NameError: name 'make_dot' is not defined

In [ ]:
# make_dot(yhat, params=dict(model_test.named_parameters()), show_attrs=True, show_saved=True).render("model_torchviz", format="png")

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_data.pos_edge_label_index)
    """
    Runs the encoder and computes node-wise latent variables
    """
    loss = model.recon_loss(z, train_data.pos_edge_label_index, train_data.neg_edge_label_index)
    """
    Given __latent variables :obj:`z`__, computes the binary cross
    entropy loss for __positive edges :obj:`pos_edge_index`__ and 
    negative sampled edges.
        Args:
            z (Tensor): The latent space :math:`\mathbf{Z}`.
            pos_edge_index (LongTensor): The positive edges to train against.
            neg_edge_index (LongTensor, optional): The negative edges to train
                against. If not given, uses negative sampling to calculate
                negative edges. (default: :obj:`None`)
    BCE: what is the probability of the latent vars to be a positive or negative edge
    """
    
    #loss = loss + (1 / data.num_nodes) * model.kl_loss()
    
    """
    kl_loss(self, mu=None, logstd=None):
        Computes the KL loss, either for the passed arguments :obj:`mu`
        and :obj:`logstd`, or based on latent variables from last encoding.
        Args:
            mu (Tensor, optional): The latent space for :math:`\mu`. If set to
                :obj:`None`, uses the last computation of :math:`mu`.
                (default: :obj:`None`)
            logstd (Tensor, optional): The latent space for
                :math:`\log\sigma`.  If set to :obj:`None`, uses the last
                computation of :math:`\log\sigma^2`.(default: :obj:`None`)
    """
    loss.backward()
    optimizer.step()
    return float(loss)


[reasoning behind loss + ...](https://github.com/tkipf/gae/issues/20) : cross-entropy loss applies to N^2 terms, i.e. all potential edges, while the KL term only applies to N terms, i.e. all nodes. This normalization makes sure they have a comparable scale.

However, should even get reasonable results with beta=0 (i.e. no KL
term at all), as the model is typically sufficiently regularized via
negative sampling (i.e. we choose random negative links).

In [ ]:
def test(pos_edge_label_index, neg_edge_label_index):
    """
    Given 
        latent variables :obj:`z`, 
        positive edges:obj:`pos_edge_index`; and 
        negative edges :obj:`neg_edge_index`,
    computes area under the ROC curve (AUC) and average precision (AP) scores and
    return latent variables, z.
    Args:
        z (Tensor): The latent space :math:`\mathbf{Z}`.
        pos_edge_index (LongTensor): The positive edges to evaluate against.
        neg_edge_index (LongTensor): The negative edges to evaluate against.
    """
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_data.pos_edge_label_index)
        results = model.test(z, pos_edge_label_index, neg_edge_label_index)
    return results[0], results[1], z #return area under curve, avg precision, and latent embeddings Z

In [ ]:
test_data.pos_edge_label_index

In [ ]:
Z = []
AUC = []
AP = []
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap, z = test(test_data.pos_edge_label_index, test_data.neg_edge_label_index)
    Z.append(z)
    AUC.append(auc)
    AP.append(ap)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    print(z)

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (15, 5))
ax[0].plot(AP)
ax[0].set_title('Avg precision')
ax[1].plot(AUC)
ax[1].set_title('Area under curve') 

Average Precision is a single number used to summarise a [Precision-Recall curve](https://jonathan-hui.medium.com/map-mean-average-precision-for-object-detection-45c121a31173). The AP is finding the area under the precision-recall curve

[AUC](https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5) tells how much the model is capable of distinguishing between classes. Higher the AUC, the better the model is at predicting 0 classes as 0 and 1 classes as 1. By analogy, the Higher the AUC, the better the model is at distinguishing between patients with the disease and no disease.

In [ ]:
max_ap = max(AP)
print('Maximum value:', max_ap, "At index:", AP.index(max_ap))
max_auc = max(AUC)
print('Maximum value:', max_auc, "At index:", AUC.index(max_auc))

In [ ]:
num = 200
print('AUC: {:.4f}, AP: {:.4f}'.format(AUC[num], AP[num]))

In [ ]:
num = 250
print('AUC: {:.4f}, AP: {:.4f}'.format(AUC[num], AP[num]))